In [1]:
import numpy_financial as npf
import datetime
def calculate_ytm(coupon, face_value, years_to_maturity, price, coupon_freq):
    coupon = coupon / 100.0
    periods = years_to_maturity *coupon_freq
    ytm = npf.rate(nper=periods, pmt=coupon*face_value/coupon_freq, pv=-price, fv=face_value)* coupon_freq
    return ytm * 100.0

def calculate_accrued_interest(coupon:float, face_value:float, years_to_maturity:float, payment_frequency:int):
    coupon = coupon / 100.0
    periods = int(years_to_maturity * payment_frequency)
    days_in_period = int(365 / payment_frequency)
    days_since_last_coupon = datetime.datetime.now().timetuple().tm_yday % days_in_period
    coupon_payment = (face_value * coupon) / payment_frequency
    accrued_interest = coupon_payment * days_since_last_coupon / days_in_period
    return accrued_interest

def calculate_dirty_price(clean_price:float, accrued_interest:float):
    dirty_price = clean_price + accrued_interest
    return dirty_price

def  calculate_time_to_maturity(maturity:datetime):
    return (maturity.date() - datetime.date.today()).days / 365

In [2]:
bond_source_url = 'https://www.hl.co.uk/shares/corporate-bonds-gilts/bond-prices/uk-gilts?column=coupon&order=desc'


In [3]:
import pandas as pd

# Set the URL to extract the data from
url = bond_source_url
# Use pandas to extract the tables from the URL
tables = pd.read_html(url)
# Select the first table, which contains the bond data
bond_data = tables[0]
# Print the first 10 rows of the bond data
print(bond_data.head(10))


                                              Issuer  Coupon (%)  \
0  Treasury 6% 07/12/2028 GBP | GB0002404191 | 02...        6.00   
1  Treasury 5% 07/03/2025 GBP | GB0030880693 | 30...        5.00   
2  Treasury 4.75% 07/12/2030 GBP | GB00B24FF097 |...        4.75   
3  Treasury 4.75% 07/12/2038 GBP | GB00B00NY175 |...        4.75   
4  Treasury 4.5% 07/09/2034 GBP | GB00B52WS153 | ...        4.50   
5  Treasury 4.5% 07/06/2028 GBP | GB00BMF9LG83 | ...        4.50   
6  Treasury 4.5% 07/12/2042 GBP | GB00B1VWPJ53 | ...        4.50   
7  Treasury 4.25% 07/12/2040 GBP | GB00B6460505 |...        4.25   
8  Treasury 4.25% 07/09/2039 GBP | GB00B3KJDS62 |...        4.25   
9  Treasury 4.25% 07/06/2032 GBP | GB0004893086 |...        4.25   

           Maturity   Price                  Actions  
0   7 December 2028  107.82  View factsheet Deal now  
1      7 March 2025  100.23  View factsheet Deal now  
2   7 December 2030  103.78  View factsheet Deal now  
3   7 December 2038  104.32  Vi

In [4]:
bond_data.columns


Index(['Issuer', 'Coupon (%)', 'Maturity', 'Price', 'Actions'], dtype='object')

In [5]:
import pandas as pd

url = "https://www.hl.co.uk/shares/corporate-bonds-gilts/bond-prices/uk-gilts?column=coupon&order=desc"

bond_data = pd.read_html(url)[0]

bond_data['ShortName'] = bond_data['Issuer'].apply(lambda x: x.split('|')[1].strip())
bond_data["Maturity"] = pd.to_datetime(bond_data["Maturity"], format="%d %B %Y")
bond_data['Ttm'] = bond_data["Maturity"].apply(lambda maturity: calculate_time_to_maturity(maturity))
bond_data['CouponFreq']=2
bond_data=bond_data.rename(columns={'Coupon (%)': 'Coupon'})
bond_data=bond_data.drop(columns=['Issuer','Actions'])
col = bond_data.pop('ShortName')
bond_data.insert(0, 'ShortName', col)

print(bond_data.head())


      ShortName  Coupon   Maturity   Price        Ttm  CouponFreq
0  GB0002404191    6.00 2028-12-07  107.82   5.389041           2
1  GB0030880693    5.00 2025-03-07  100.23   1.632877           2
2  GB00B24FF097    4.75 2030-12-07  103.78   7.389041           2
3  GB00B00NY175    4.75 2038-12-07  104.32  15.394521           2
4  GB00B52WS153    4.50 2034-09-07  102.38  11.142466           2


In [6]:
bond_data.head

<bound method NDFrame.head of        ShortName  Coupon   Maturity    Price        Ttm  CouponFreq
0   GB0002404191   6.000 2028-12-07  107.820   5.389041           2
1   GB0030880693   5.000 2025-03-07  100.230   1.632877           2
2   GB00B24FF097   4.750 2030-12-07  103.780   7.389041           2
3   GB00B00NY175   4.750 2038-12-07  104.320  15.394521           2
4   GB00B52WS153   4.500 2034-09-07  102.380  11.142466           2
..           ...     ...        ...      ...        ...         ...
65  GB00BL68HJ26   0.125 2026-01-30   89.520   2.534247           2
66  GB00B4JYZV64   0.000 2034-09-07   74.100  11.142466           2
67  GB0030884786   0.000 2025-03-07   92.800   1.632877           2
68  GB00B1HYR000   0.000 2027-12-07   84.400   4.386301           2
69  GB0009140269   0.000 2029-12-07   82.372   6.389041           2

[70 rows x 6 columns]>

In [7]:
bond_data.columns


Index(['ShortName', 'Coupon', 'Maturity', 'Price', 'Ttm', 'CouponFreq'], dtype='object')

In [8]:
FACE_VALUE=100.0

In [9]:
bond_data.head(1)

,ShortName,Coupon,Maturity,Price,Ttm,CouponFreq
0,GB0002404191,6.0,2028-12-07,107.82,5.389041,2


In [10]:
bond_data['AccruedInterest']=bond_data.apply(lambda x: calculate_accrued_interest(x['Coupon'],FACE_VALUE,x['Ttm'],x['CouponFreq']),axis=1)
bond_data['DirtyPrice']=bond_data.apply(lambda x: calculate_dirty_price(x['Price'],x['AccruedInterest']),axis=1)
bond_data['DirtyYield'] = bond_data.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['DirtyPrice'],x['CouponFreq']), axis=1)
bond_data['CleanYield'] = bond_data.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['Price'],x['CouponFreq']), axis=1)

In [11]:
bond_data.dtypes


ShortName                  object
Coupon                    float64
Maturity           datetime64[ns]
Price                     float64
Ttm                       float64
CouponFreq                  int64
AccruedInterest           float64
DirtyPrice                float64
DirtyYield                float64
CleanYield                float64
dtype: object

In [12]:
bond_data=bond_data.sort_values('Ttm')


In [13]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

bond_data_styler=bond_data.style.set_properties(**{'font-size': '6pt'})
bond_data_styler

,ShortName,Coupon,Maturity,Price,Ttm,CouponFreq,AccruedInterest,DirtyPrice,DirtyYield,CleanYield
48,GB00BF0HZ991,0.750000,2023-07-22 00:00:00,99.838000,0.005479,2,0.039148,99.877148,24.534150,32.697261
27,GB00B7Z53659,2.250000,2023-09-07 00:00:00,99.630000,0.134247,2,0.117445,99.747445,4.156009,5.050128
61,GB00BMGR2791,0.125000,2024-01-31 00:00:00,97.480000,0.534247,2,0.006525,97.486525,4.950177,4.963016
43,GB00BFWFPL34,1.000000,2024-04-22 00:00:00,97.030000,0.758904,2,0.052198,97.082198,4.965173,5.037907
26,GB0008983024,2.500000,2024-07-17 00:00:00,375.040000,0.994521,2,0.130495,375.170495,-96.144235,-96.126011
24,GB00BHBFH458,2.750000,2024-09-07 00:00:00,97.440000,1.136986,2,0.143544,97.583544,4.962084,5.095981
58,GB00BLPK7110,0.250000,2025-01-31 00:00:00,93.200000,1.536986,2,0.013049,93.213049,4.887393,4.896737
1,GB0030880693,5.000000,2025-03-07 00:00:00,100.230000,1.632877,2,0.260989,100.490989,4.684159,4.851790
67,GB0030884786,0.000000,2025-03-07 00:00:00,92.800000,1.632877,2,0.000000,92.800000,4.628946,4.628946
52,GB00BK5CVX03,0.625000,2025-06-07 00:00:00,92.530000,1.884932,2,0.032624,92.562624,4.798969,4.818207


In [14]:
bond_data.to_excel(r"c:\temp\bond_data.xlsx", index=False)

In [15]:
# yield filtering 
df_less_than_Ttm = bond_data[bond_data['Ttm']<20]

In [16]:
df_less_than_Ttm=df_less_than_Ttm.sort_values("DirtyYield", ascending=False)
cols = ['ShortName', 'Coupon', 'Maturity', 'Price', 'Ttm','DirtyPrice', 'DirtyYield', 'CleanYield']
df_less_than_Ttm = df_less_than_Ttm[cols]
df_less_than_Ttm


,ShortName,Coupon,Maturity,Price,Ttm,DirtyPrice,DirtyYield,CleanYield
48,GB00BF0HZ991,0.750,2023-07-22,99.838,0.005479,99.877148,24.534150,32.697261
19,GB00BPCJD880,3.500,2025-10-22,96.630,2.260274,96.812692,5.009042,5.097410
43,GB00BFWFPL34,1.000,2024-04-22,97.030,0.758904,97.082198,4.965173,5.037907
24,GB00BHBFH458,2.750,2024-09-07,97.440,1.136986,97.583544,4.962084,5.095981
61,GB00BMGR2791,0.125,2024-01-31,97.480,0.534247,97.486525,4.950177,4.963016
58,GB00BLPK7110,0.250,2025-01-31,93.200,1.536986,93.213049,4.887393,4.896737
29,GB00BTHH2R79,2.000,2025-09-07,94.210,2.136986,94.314396,4.832293,4.886277
52,GB00BK5CVX03,0.625,2025-06-07,92.530,1.884932,92.562624,4.798969,4.818207
1,GB0030880693,5.000,2025-03-07,100.230,1.632877,100.490989,4.684159,4.851790
15,GB00BL6C7720,4.125,2027-01-29,98.050,3.531507,98.265316,4.663452,4.731075
